### https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

In [1]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
reviews_test = []
reviews_train = []

for line in open('data/aclImdb/movie_data/full_train.txt', encoding="utf8"):
    reviews_train.append(line.strip())
reviews_train[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [3]:
for line in open('data/aclImdb/movie_data/full_test.txt', encoding="utf8"):
    reviews_test.append(line.strip())
reviews_test[0]

"I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge."

In [4]:
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_WITH_SPACE.sub(" ", REPLACE_NO_SPACE.sub("", review.lower())) for review in reviews]
    return reviews
reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [5]:
cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [6]:
target = [1 if x < 12500 else 0 for x in range(25000)]
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size=0.75)

In [7]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, max_iter=500)
    lr.fit(X_train, y_train)
    print("Accuracy for C=%s: %s" % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.87232
Accuracy for C=0.05: 0.88128
Accuracy for C=0.25: 0.88608
Accuracy for C=0.5: 0.88336
Accuracy for C=1: 0.88096


In [8]:
final_model = LogisticRegression(C=0.05, max_iter=200)
final_model.fit(X, target)
print("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Final Accuracy: 0.88172


In [9]:
feature_to_coef = {word: coef for word, coef in zip(cv.get_feature_names(), final_model.coef_[0])}

for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(best_positive)

('excellent', 0.928930721254566)
('perfect', 0.7924073691466867)
('great', 0.6738714624335574)
('amazing', 0.6128669705071148)
('superb', 0.6007357897217929)


In [10]:
for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
    print(best_negative)

('worst', -1.364739117410738)
('waste', -1.166566080710549)
('awful', -1.03207672028567)
('poorly', -0.873759054451777)
('boring', -0.8568152889177791)
